# Tugas Pertemuan 1
Crawling berita


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Fungsi untuk mengambil data dari halaman web Detik.com
def get_data(url, kategori):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return

    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all("article", class_="list-content__item")

    for article in articles:
        if len(judul) >= 10:  # Hentikan jika sudah 10 berita
            return

        try:
            link = article.find("a")["href"]
            article_response = requests.get(link)
            article_response.raise_for_status()
        except (requests.exceptions.RequestException, TypeError) as e:
            print(f"Request for article failed: {e}")
            continue

        article_soup = BeautifulSoup(article_response.content, "html.parser")
        title_element = article_soup.find("h1", class_="detail__title")
        title = title_element.text.strip() if title_element else "Title Not Found"
        date_element = article_soup.find("div", class_="detail__date")
        date = date_element.text.strip() if date_element else "Date Not Found"
        content_element = article_soup.find("div", class_="detail__body-text")
        content = content_element.text.strip() if content_element else "Content Not Found"

        judul.append(title)
        tanggal.append(date)
        isi.append(content)

        print(title)
        time.sleep(1)  # Menambahkan jeda waktu 1 detik antara permintaan artikel

# Membuat list url dan kategori yang akan di-crawl
base_urls = ["https://sport.detik.com/indeks"]
categories = ["Olahraga"]

# Inisialisasi list untuk menyimpan data
judul = []
tanggal = []
isi = []

# Melakukan iterasi untuk setiap url dan kategori
for base_url, category in zip(base_urls, categories):
    for page in range(1, 4):  # Looping untuk beralih halaman
        if len(judul) >= 10:  # Hentikan jika sudah 10 berita
            break

        url = f"{base_url}/{page}"
        get_data(url, category)
        time.sleep(2)  # Menambahkan jeda waktu 2 detik antara permintaan halaman

# Membuat dataframe dari list data
df = pd.DataFrame({
    "judul": judul,
    "tanggal": tanggal,
    "isi": isi
})

# Menyimpan dataframe ke file CSV
df.to_csv("Crawl-berita.csv", index = False)


# Crawling berita olahraga
Crawling data atau web crawling adalah proses ekstraksi informasi dari situs web dengan menggunakan program komputer. Tujuan utama dari web crawling adalah untuk mengumpulkan data dari berbagai sumber di internet.
Terdapat berbagai macam library yang dapat digunakan untuk crawling data. Library yang digunakan untuk crawling berita Kompas yaitu Beautiful Soup dan Requests. Beautiful Soup Digunakan untuk mengekstrak informasi dari HTML dan Request digunakan untuk mengirim permintaan HTTP dan mengakses halaman web.

In [2]:
df=pd.read_csv("Crawl-berita.csv")
df

,judul,tanggal,isi
0,Marc Marquez Memang Agak Laen,"Selasa, 03 Sep 2024 17:30 WIB",Jakarta - Marc Marquez meraih kemenangan setel...
1,Daud Yordan Kembali Naik Ring: Ini Bukan Perta...,"Selasa, 03 Sep 2024 17:17 WIB","Jakarta - Petinju dunia asal Indonesia, Daud '..."
2,"Kontroversi Valentina Petrillo, Atlet Transgen...","Selasa, 03 Sep 2024 16:30 WIB",Paris - Valentina Petrillo menjadi kontroversi...
3,Jonatan hingga Gregoria Bakal Comeback di Hong...,"Selasa, 03 Sep 2024 15:52 WIB",Jakarta - Atlet-atlet bulutangkis andalan Indo...
4,Ranking BWF: Leo/Bagas Meroket 85 Tingkat Usai...,"Selasa, 03 Sep 2024 14:30 WIB",Jakarta - Inilah ranking Badminton World Feder...
5,Leo/Bagas dan Fikri/Faniel Akan Diuji Saat Pem...,"Selasa, 03 Sep 2024 13:15 WIB",Jakarta - Tren positif Leo Rolly Carnando/Baga...
6,"Evaluasi Pelatih Usai Leo/Bagas Juara, Fikri/D...","Selasa, 03 Sep 2024 12:30 WIB",Jakarta - Pelatih ganda putra Aryono Miranat m...
7,"Lolos Semifinal US Open 2024, Aldila Sutjiadi ...","Selasa, 03 Sep 2024 09:30 WIB",New York - Aldila Sutjiadi sukses melangkah ke...
8,Jete Run 2024 untuk Kampanye Hidup Sehat,"Selasa, 03 Sep 2024 02:15 WIB",Surabaya - Ajang Jete Run 2024 sukses digelar ...
9,Dukungan untuk KONI Agar PON 2024 Sukses,"Selasa, 03 Sep 2024 01:15 WIB",Jakarta - Menyelenggarakan Pekan Olahraga Nasi...
